In [53]:

import cvxpy as cvx
import numpy as np
from scipy.sparse import eye, csr_matrix, hstack, linalg
from scipy.signal import argrelextrema

def gen_d2(n):
    """
    Generate the 2nd difference matrix.
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I2 = eye(n - 2)
    O2 = csr_matrix((n - 2, 1))
    return hstack((I2, O2, O2)) + hstack((O2, -2 * I2, O2)) + hstack((O2, O2, I2))


def gen_d1(n):
    """
    Generate the 1st difference matrix
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I1 = eye(n - 1)
    O1 = csr_matrix((n - 1, 1))
    return hstack((I1, O1)) - hstack((O1, I1))


def get_max_lam(y):
    """
    Calculate the max lambda value for given time series y
    :param y: np.array, time series given
    :return: float, max lambda value
    """
    D = gen_d2(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def get_max_rho(y):
    """
    Calculate the max rho value for given time series y,
    :param y: np.array, time series given
    :return: float, max rho value
    """
    D = gen_d1(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def l1filter(t, y,
             lam=1200,
             rho=80,
             periods=(365.25, 182.625),
             solver=cvx.MOSEK,
             verbose=False):
    """
    Do l1 regularize for given time series.
    :param t: np.array, time
    :param y: np.array, time series value
    :param lam: lambda value
    :param rho: rho value
    :param periods: list, periods, same unit as t
    :param solver: cvx.solver
    :param verbose: bool, show verbose or not
    :return: x, w, s, if periods is not None, else return x, w
    """
    t = np.asarray(t, dtype=np.float64)
    t = t - t[0]
    y = np.asarray(y, dtype=np.float64)

    assert y.shape == t.shape

    n = len(t)
    D = gen_d2(n)

    x = cvx.Variable(n)
    w = cvx.Variable(n)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal
    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) +
#                       lam * cvx.sum_squares(D * x) +
#                       lam * cvx.norm(D * x, 2) +
                       lam * cvx.norm(D * x, 1) +
                       rho * cvx.tv(w))
    prob = cvx.Problem(obj)
    print('solving 1')
    prob.solve(solver=solver, verbose=verbose)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None

    t = np.asarray(t, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    n = len(t)
    x = cvx.Variable(n)
    w = cvx.Variable(n)
    dx = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(x))
    x_term = cvx.tv(dx)
    dw = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(w))
    w_term = cvx.norm(dw, 1)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal

    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) + lam * x_term + rho * w_term)
    prob = cvx.Problem(obj)
    print('solving')
    prob.solve(solver=solver, verbose=verbose)
    print(x)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None




In [94]:

from pyacs.gts.Sgts import Sgts
import matplotlib.pyplot as plt
%matplotlib qt

ts_dir = '/Users/nocquet/Dropbox/sse_project/ecuador/pck'
ts = Sgts(ts_dir, verbose=False)
#ts.CABP.plot()
mts = ts.HSPR.extract_periods([2000,2016.29]).detrend_median(periods=[2000,2016.29])
#.extract_periods([2015.5,2016.5])
t = mts.data[:,0]
y = mts.data[:,2]*1E3
x,w,s = l1filter(t,y,lam=300,rho=30,periods=None,verbose=True)
print(s)

plt.plot(t,y,label='y');plt.plot(t,x,label='filtered');plt.plot(t,w,label='offset');plt.legend()
#plt.plot(s,label='seas');



solving 1


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 13061           
  Cones                  : 1               
  Scalar variables       : 13065           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 13061           
  Cones                  : 1               
  Scalar variables       : 13065           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 8               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 5223
Optimizer  - Cones                  : 1
Optimizer  - Scala

/usr/local/geodesy/anaconda38/lib/python3.8/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
